# Resume Scoring wtih Dummy sample data

In [ ]:
import streamlit as st
import pandas as pd

# --- Role and Skill Weight Configuration ---
job_roles = pd.DataFrame({
    'job_role': ['Data Analyst', 'Data Engineer', 'BI Developer'],
    'base_weight': [70, 85, 80]
})

job_levels = pd.DataFrame({
    'job_level': ['Junior', 'Mid', 'Senior'],
    'level_weight': [0.5, 1, 1.5]
})

# Role-specific skill weight table
skills = pd.DataFrame({
    'skill': [
        'SQL', 'Python', 'Machine Learning', 'Data Visualization', 
        'Power BI', 'Tableau', 'ETL Tools', 'Big Data', 'Statistics', 'Excel'
    ],
    'Data Analyst':     [20, 15, 10, 20, 25, 20, 10, 5, 15, 20],
    'Data Engineer':    [25, 20, 15, 10, 10, 5, 25, 20, 10, 10],
    'BI Developer':     [15, 10, 10, 25, 30, 25, 10, 5, 10, 15]
})

experience = pd.DataFrame({
    'years_of_experience': [0, 1, 3, 5, 8],
    'experience_weight': [0, 10, 20, 30, 40]
})

# Updated user data with role-appropriate skills
user_data = pd.DataFrame({
    'full_name': ['Alice', 'Bob', 'Charlie', 'David'],
    'job_role': ['Data Analyst', 'Data Engineer', 'BI Developer', 'Data Analyst'],
    'job_level': ['Junior', 'Mid', 'Senior', 'Mid'],
    'skills': [
        ['SQL', 'Power BI', 'Excel'],                    # Alice - DA
        ['SQL', 'Python', 'ETL Tools', 'Big Data'],      # Bob - DE
        ['Power BI', 'Data Visualization', 'Tableau'],   # Charlie - BI Dev
        ['SQL', 'Excel', 'Python', 'Statistics']         # David - DA
    ],
    'years_of_experience': [1, 5, 3, 2]
})

# --- Helper Functions ---
def get_weight(df, column, value, weight_col):
    row = df[df[column] == value]
    return row[weight_col].values[0] if not row.empty else 0

def get_role_specific_skill_weight(skill, job_role):
    row = skills[skills['skill'] == skill]
    return row[job_role].values[0] if not row.empty else 0

def calculate_weight(job_role, job_level, selected_skills, years_exp):
    role_weight = get_weight(job_roles, 'job_role', job_role, 'base_weight')
    level_weight = get_weight(job_levels, 'job_level', job_level, 'level_weight')
    skill_weight = sum([get_role_specific_skill_weight(s, job_role) for s in selected_skills])
    exp_weight = get_weight(experience, 'years_of_experience', years_exp, 'experience_weight')
    #take out the job level weight
    total = role_weight + (role_weight * level_weight) + skill_weight + exp_weight
    #total = role_weight + skill_weight + exp_weight
    return round(total, 2)

def candidate_score(row, selected_job, selected_level, selected_skills, selected_exp):
    role_weight = get_weight(job_roles, 'job_role', selected_job, 'base_weight')

    # Job level comparison: candidate level must be >= selected level
    candidate_level = row['job_level']
    candidate_level_weight = 0
    if job_levels.index[job_levels['job_level'] == candidate_level].item() >= \
       job_levels.index[job_levels['job_level'] == selected_level].item():
        candidate_level_weight = get_weight(job_levels, 'job_level', candidate_level, 'level_weight')

    # Skills
    matched_skills = set(row['skills']).intersection(set(selected_skills))
    skill_weight = sum([
        get_role_specific_skill_weight(s, selected_job) for s in matched_skills
    ])

    # Experience
    experience_weight = 0
    if row['years_of_experience'] >= selected_exp:
        experience_weight = get_weight(experience, 'years_of_experience', row['years_of_experience'], 'experience_weight')
    
    #take out job level weight 
    total = role_weight + (role_weight * candidate_level_weight) + skill_weight + experience_weight
    #total = role_weight  + skill_weight + experience_weight
    return round(total)

# --- Streamlit App ---
st.title("📊 ATS Resume Ranking System")

# Employer Input
selected_job = st.selectbox("Select Job Role", job_roles['job_role'])
selected_level = st.selectbox("Select Job Level", job_levels['job_level'])
selected_skills = st.multiselect("Select Required Skills", skills['skill'])
selected_exp = st.selectbox("Select Minimum Years of Experience", experience['years_of_experience'])

# Baseline Score Calculation
baseline_weight = calculate_weight(selected_job, selected_level, selected_skills, selected_exp)
st.markdown(f"### Baseline Competency Weight: `{baseline_weight}`")

# Candidate Evaluation
user_data['competency_weight'] = user_data.apply(
    lambda row: candidate_score(row, selected_job, selected_level, selected_skills, selected_exp),
    axis=1
)

user_data['status'] = user_data['competency_weight'].apply(
    lambda x: "✓" if x >= baseline_weight else "✗"
)

# Sort Candidates by Score
ranked_users = user_data.sort_values(by='competency_weight', ascending=False)

# Styling for Status
def highlight_status(val):
    color = 'green' if val == '✓' else 'red'
    return f'color: {color}; font-weight: bold;'

styled_df = ranked_users[[
    'full_name', 'job_role', 'job_level', 'skills', 'years_of_experience', 
    'competency_weight', 'status'
]].style.applymap(highlight_status, subset=['status'])

st.subheader("🏅 Ranked Candidates")
st.dataframe(styled_df, use_container_width=True)


# Resume Scoring with Snowfalke Database 

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

# Connect to Snowflake
session = st.connection('snowflake').session()

# SQL query to fetch candidate data from Snowflake
def fetch_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE
    FROM CANDIDATE_PROFILES
    """
    df = session.sql(query).to_pandas()
    return df

# Fetch data from Snowflake
user_data = fetch_data()

# Job Roles with base weights
job_roles = pd.DataFrame({
    'job_role': ['Data Analyst', 'BI Consultant', 'Data Scientist', 'Data Engineer'],
    'base_weight': [70, 80, 85, 90]  # Example base weights for each job role
})

# Job Levels with weight multipliers
job_levels = pd.DataFrame({
    'job_level': ['Junior', 'Mid-level', 'Senior'],
    'level_weight': [0.5, 1, 1.5]  # Weight for different job levels
})

# Skills and role-specific weights
skills = pd.DataFrame({
    'skill': [
        'AWS', 'Apache Airflow', 'Apache Kafka', 'Apache Spark', 'DAX', 'Excel', 'Hadoop', 
        'NLP', 'Power BI', 'Python', 'R', 'SQL', 'Tableau', 'TensorFlow'
    ],
    'Data Analyst': [5, 0, 0, 0, 20, 15, 0, 0, 25, 15, 0, 20, 20, 0],
    'BI Consultant': [0, 10, 5, 5, 20, 10, 10, 5, 25, 15, 10, 15, 25, 10],
    'Data Scientist': [10, 15, 15, 10, 25, 15, 10, 10, 15, 20, 20, 20, 15, 30],
    'Data Engineer': [20, 20, 25, 25, 10, 10, 20, 15, 10, 20, 10, 25, 5, 15]
})

# Years of Experience with corresponding weights
experience = pd.DataFrame({
    'years_of_experience': [1, 2, 3, 4, 5, 6, 7],
    'experience_weight': [10, 10, 20, 20, 30, 30, 40]  # Weights assigned based on years of experience
})

# --- Helper Functions ---
def get_weight(df, column, value, weight_col):
    row = df[df[column] == value]
    return row[weight_col].values[0] if not row.empty else 0

def get_role_specific_skill_weight(skill, job_role):
    row = skills[skills['skill'] == skill]
    return row[job_role].values[0] if not row.empty else 0

def calculate_weight(job_role, job_level, selected_skills, years_exp):
    role_weight = get_weight(job_roles, 'job_role', job_role, 'base_weight')
    level_weight = get_weight(job_levels, 'job_level', job_level, 'level_weight')
    skill_weight = sum([get_role_specific_skill_weight(s, job_role) for s in selected_skills])
    exp_weight = get_weight(experience, 'years_of_experience', years_exp, 'experience_weight')
    total = role_weight + (role_weight * level_weight) + skill_weight + exp_weight
    return round(total, 2)

def candidate_score(row, selected_job, selected_level, selected_skills, selected_exp):
    role_weight = get_weight(job_roles, 'job_role', selected_job, 'base_weight')

    # Job level comparison: candidate level must be >= selected level
    candidate_level = row['JOB_LEVEL']
    candidate_level_weight = 0
    candidate_level_index = job_levels.index[job_levels['job_level'] == candidate_level].item()
    selected_level_index = job_levels.index[job_levels['job_level'] == selected_level].item()
    if candidate_level_index >= selected_level_index:
        candidate_level_weight = get_weight(job_levels, 'job_level', candidate_level, 'level_weight')

    # Skills
    matched_skills = set(row['TECHNICAL_SKILLS'].split(',')).intersection(set(selected_skills))  # Assuming skills are stored as a comma-separated string
    skill_weight = sum([
        get_role_specific_skill_weight(s, selected_job) for s in matched_skills
    ])

    # Experience
    experience_weight = 0
    if row['YEARS_OF_EXPERIENCE'] >= selected_exp:
        experience_weight = get_weight(experience, 'years_of_experience', row['YEARS_OF_EXPERIENCE'], 'experience_weight')
    
    total = role_weight + (role_weight * candidate_level_weight) + skill_weight + experience_weight
    return round(total)

# --- Streamlit App ---
st.title("📊 ATS Resume Ranking System")

# Employer Input
selected_job = st.selectbox("Select Job Role", job_roles['job_role'])
selected_level = st.selectbox("Select Job Level", job_levels['job_level'])
selected_skills = st.multiselect("Select Required Skills", skills['skill'])
selected_exp = st.selectbox("Select Minimum Years of Experience", experience['years_of_experience'])

# Baseline Score Calculation
baseline_weight = calculate_weight(selected_job, selected_level, selected_skills, selected_exp)
st.markdown(f"### Baseline Competency Weight: `{baseline_weight}`")

# Candidate Evaluation
user_data['competency_weight'] = user_data.apply(
    lambda row: candidate_score(row, selected_job, selected_level, selected_skills, selected_exp),
    axis=1
)

user_data['status'] = user_data['competency_weight'].apply(
    lambda x: "✓" if x >= baseline_weight else "✗"
)

# Sort Candidates by Score
ranked_users = user_data.sort_values(by='competency_weight', ascending=False)

# Styling for Status
def highlight_status(val):
    color = 'green' if val == '✓' else 'red'
    return f'color: {color}; font-weight: bold;'

styled_df = ranked_users[[
    'FULL_NAME', 'PHONE_NUMBER', 'EMAIL', 'LOCATION', 'JOB_TITLE', 'JOB_LEVEL', 'TECHNICAL_SKILLS', 'YEARS_OF_EXPERIENCE', 
    'competency_weight', 'status'
]].style.applymap(highlight_status, subset=['status'])

st.subheader("🏅 Ranked Candidates")
st.dataframe(styled_df, use_container_width=True)